In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from haversine import haversine

In [2]:
df_military=pd.read_excel("./Military.xlsx")
df_military_location=df_military[['COMPONENT','SITE_NAME','Address','City','State','Zip','COUNTRY','Latitude','Longitude']]
df_military_location=df_military_location.rename(columns={"Zip":"zip_cd"})
df_military_location=df_military_location.rename(columns={"Latitude":"latitude"})
df_military_location=df_military_location.rename(columns={"Longitude":"longitude"})
df_military_location['zip_cd']=df_military_location['zip_cd'].apply(lambda x: str(int(x)).zfill(5) if str(x)!="nan" else x)

df_military_location.head(2)

,COMPONENT,SITE_NAME,Address,City,State,zip_cd,COUNTRY,latitude,longitude
0,AF Guard,Luis Munoz Marin IAP,17 Cll 2,Carolina,PR,00983,Puerto Rico,18.443918,-65.994070
1,AF Guard,NAS JRB New Orleans,400Russell Dr,Belle Chasse,LA,70037,United States,29.834335,-90.019897


In [3]:
df_hospitals=pd.read_csv("./Hospitals.csv")
df_hospital_location=df_hospitals[['NAME','ADDRESS','CITY','STATE','ZIP','Y','X']]
df_hospital_location=df_hospital_location.rename(columns={"ZIP":"zip_cd"})
df_hospital_location=df_hospital_location.rename(columns={"X":"longitude"})
df_hospital_location=df_hospital_location.rename(columns={"Y":"latitude"})
df_hospital_location['zip_cd']=df_hospital_location['zip_cd'].apply(lambda x: str(int(x)).zfill(5) if str(x)!="nan" else x)


df_hospital_location.head(2)

,NAME,ADDRESS,CITY,STATE,zip_cd,latitude,longitude
0,COLORADO RIVER MEDICAL CENTER,1401 BAILEY AVENUE,NEEDLES,CA,92363,34.833066,-114.617494
1,ALVARADO HOSPITAL MEDICAL CENTER,6655 ALVARADO ROAD,SAN DIEGO,CA,92120,32.776586,-117.057065


In [4]:
df_BL_store=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",sep="|",usecols=['location_id','latitude_meas','longitude_meas'])
print(df_BL_store.shape)
df_BL_store=df_BL_store[~df_BL_store['location_id'].isin([145,6990])]
print(df_BL_store.shape)

dict_BL_store=df_BL_store.copy()
dict_BL_store['store_loc']=dict_BL_store[['latitude_meas','longitude_meas']].values.tolist()
dict_BL_store=dict_BL_store.set_index('location_id').to_dict()['store_loc']
dict_BL_store

(1406, 3)
(1404, 3)


{3: [33.42157, -82.01892099999999],
 30: [41.363132, -83.650909],
 43: [41.612553000000005, -83.623582],
 49: [39.178574, -84.281631],
 58: [38.190715999999995, -83.47895799999999],
 64: [33.050872999999996, -85.02681700000001],
 77: [40.579575, -83.086617],
 83: [41.432655, -82.187011],
 91: [39.685836, -86.135239],
 109: [39.099323999999996, -84.483737],
 117: [32.814445, -79.997371],
 132: [40.397225, -86.871866],
 135: [37.7036, -85.8653],
 153: [39.25928, -84.80390799999999],
 180: [36.961983000000004, -86.468292],
 190: [37.8333, -87.5738],
 200: [40.606633, -79.600773],
 205: [37.100303000000004, -84.623106],
 209: [41.7971, -86.247574],
 214: [39.761463, -89.60889399999999],
 219: [33.18615, -80.001221],
 222: [36.418927000000004, -82.992693],
 225: [36.529346999999994, -82.525337],
 228: [40.834444, -77.80412],
 235: [38.729903, -90.380777],
 244: [38.714363, -82.858125],
 246: [39.281929, -80.276009],
 251: [39.80247, -85.77176],
 255: [38.7704, -85.416542],
 257: [38.861077,

In [5]:
df_military_location['nearest_BL_store']=np.nan
df_military_location['nearest_dist_to_BL']=np.nan

for ind,row in df_military_location.iterrows():
    loc=(row['latitude'],row['longitude'])
    min_dist=np.inf
    nearest_store=np.nan
    for BL_num, v in dict_BL_store.items():
        dist=haversine(loc,v,unit="mi")
        if dist<=min_dist:
            min_dist=dist
            nearest_store=BL_num
    df_military_location.loc[ind,"nearest_BL_store"]=nearest_store
    df_military_location.loc[ind,"nearest_dist_to_BL"]=min_dist
df_military_location_25=df_military_location[df_military_location['nearest_dist_to_BL']<=25]

In [6]:
df_hospital_location['nearest_BL_store']=np.nan
df_hospital_location['nearest_dist_to_BL']=np.nan

for ind,row in df_hospital_location.iterrows():
    loc=(row['latitude'],row['longitude'])
    min_dist=np.inf
    nearest_store=np.nan
    for BL_num, v in dict_BL_store.items():
        dist=haversine(loc,v,unit="mi")
        if dist<=min_dist:
            min_dist=dist
            nearest_store=BL_num
    df_hospital_location.loc[ind,"nearest_BL_store"]=nearest_store
    df_hospital_location.loc[ind,"nearest_dist_to_BL"]=min_dist
df_hospital_location_25=df_hospital_location[df_hospital_location['nearest_dist_to_BL']<=25]

In [7]:
combined_unique_zips_25=df_hospital_location_25[['zip_cd']].append(df_military_location_25[['zip_cd']]).drop_duplicates()
combined_unique_zips_25=combined_unique_zips_25[pd.notnull(combined_unique_zips_25['zip_cd'])]
combined_unique_zips_25.shape

(4357, 1)

In [8]:
DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2],skiprows=1)
DMA_zip.columns=['zip_cd','DMA']
DMA_zip=DMA_zip.drop_duplicates()
DMA_zip=DMA_zip.groupby("zip_cd")['DMA'].agg(list).reset_index()
DMA_zip=DMA_zip.rename(columns={"DMA":"DMA_list"})
DMA_zip.head(2)

,zip_cd,DMA_list
0,00501,[NEW YORK]
1,00544,[NEW YORK]


In [9]:
max_len_DMA=DMA_zip['DMA_list'].apply(len).max()
max_len_DMA

4

In [10]:
combined_unique_zips_25=pd.merge(combined_unique_zips_25,DMA_zip,on="zip_cd",how="left")

combined_unique_zips_25['DMA_list']=combined_unique_zips_25['DMA_list'].astype(str)
combined_unique_zips_25['DMA_list']=combined_unique_zips_25['DMA_list'].replace("nan","['nan']")
combined_unique_zips_25['DMA_list']=combined_unique_zips_25['DMA_list'].apply(eval)
combined_unique_zips_25['DMA_1']=combined_unique_zips_25['DMA_list'].apply(lambda x: x[0] if len(x)>=1 else np.nan)
combined_unique_zips_25['DMA_2']=combined_unique_zips_25['DMA_list'].apply(lambda x: x[1] if len(x)>=2 else np.nan)
combined_unique_zips_25['DMA_3']=combined_unique_zips_25['DMA_list'].apply(lambda x: x[2] if len(x)>=3 else np.nan)
combined_unique_zips_25['DMA_4']=combined_unique_zips_25['DMA_list'].apply(lambda x: x[4] if len(x)>=4 else np.nan)

In [11]:
writer=pd.ExcelWriter("./BL_bighero_zips_with_locations_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
combined_unique_zips_25.to_excel(writer,"unique_zips_both",index=False)
df_hospital_location_25.to_excel(writer,"hospital_in_25_miles",index=False)
df_military_location_25.to_excel(writer,"military_in_25_miles",index=False)
df_hospital_location.to_excel(writer,"all_hospitals",index=False)
df_military_location.to_excel(writer,"all_military",index=False)

# df_hospitals.to_excel(writer,"input_hospitals",index=False)
# df_military.to_excel(writer,"input_military",index=False)

writer.save()